In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
target = np.array(data['Survived'])
data.drop('Survived', axis=1, inplace=True)

In [5]:
def data_prepare(data, data_source):
    data.drop('Cabin', axis=1, inplace=True)
    data.drop('Name', axis=1, inplace=True)
    data.drop('Ticket', axis=1, inplace=True)
    data.set_index('PassengerId', inplace=True)
    data['Age'] = data['Age'].fillna(data_source['Age'].mean())
    data['Sex'] = (data['Sex'] == 'female').astype(int)
    data['Pclass'] = data['Pclass'].astype('category')
    data = pd.get_dummies(data)
    return data

In [6]:
data = data_prepare(data, data)

In [7]:
data.head()

,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,1,0,0,1
2,1,38.0,1,0,71.2833,1,0,0,1,0,0
3,1,26.0,0,0,7.9250,0,0,1,0,0,1
4,1,35.0,1,0,53.1000,1,0,0,0,0,1
5,0,35.0,0,0,8.0500,0,0,1,0,0,1


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Sex           891 non-null int64
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Pclass_1      891 non-null uint8
Pclass_2      891 non-null uint8
Pclass_3      891 non-null uint8
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: float64(2), int64(3), uint8(6)
memory usage: 47.0 KB


In [9]:
data.describe()

,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.352413,29.699118,0.523008,0.381594,32.204208,0.242424,0.206510,0.551066,0.188552,0.086420,0.722783
std,0.477990,13.002015,1.102743,0.806057,49.693429,0.428790,0.405028,0.497665,0.391372,0.281141,0.447876
min,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,29.699118,0.000000,0.000000,14.454200,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1.000000,35.000000,1.000000,0.000000,31.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [11]:
parameters = [{'n_estimators': [150, 200, 250, 300], 
               'max_features': np.arange(5, 10),
               'max_depth': np.arange(4, 9)}]

In [12]:
clf = GridSearchCV(estimator=RandomForestClassifier(), 
                   param_grid=parameters,
                   scoring='accuracy',
                   cv=5)

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(data, target)

In [14]:
%%time
clf.fit(X_train, y_train)

CPU times: user 1min 21s, sys: 0 ns, total: 1min 21s
Wall time: 1min 22s


/home/ivan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [150, 200, 250, 300], 'max_features': array([5, 6, 7, 8, 9]), 'max_depth': array([4, 5, 6, 7, 8])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [15]:
clf.best_params_

{'max_depth': 4, 'max_features': 6, 'n_estimators': 200}

In [16]:
y_pred = clf.predict(X_valid)
accuracy_score(y_pred, y_valid)

0.8295964125560538

In [17]:
data_test = pd.read_csv('test.csv')
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
data_test = data_prepare(data_test, data)

In [19]:
data_test.head()

,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
892,0,34.5,0,0,7.8292,0,0,1,0,1,0
893,1,47.0,1,0,7.0000,0,0,1,0,0,1
894,0,62.0,0,0,9.6875,0,1,0,0,1,0
895,0,27.0,0,0,8.6625,0,0,1,0,0,1
896,1,22.0,1,1,12.2875,0,0,1,0,0,1


In [20]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 11 columns):
Sex           418 non-null int64
Age           418 non-null float64
SibSp         418 non-null int64
Parch         418 non-null int64
Fare          417 non-null float64
Pclass_1      418 non-null uint8
Pclass_2      418 non-null uint8
Pclass_3      418 non-null uint8
Embarked_C    418 non-null uint8
Embarked_Q    418 non-null uint8
Embarked_S    418 non-null uint8
dtypes: float64(2), int64(3), uint8(6)
memory usage: 22.0 KB


In [21]:
data_test['Fare'] = data_test['Fare'].fillna(data['Fare'].mean())

In [22]:
y_test_pred = clf.predict(data_test)

In [23]:
y_test_pred = pd.DataFrame({'Survived': y_test_pred}, index=data_test.index)
y_test_pred.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0


In [24]:
y_test_pred.to_csv('submission.csv')

### Ок
Подсмотрел пару хороших идей на Kaggle

In [26]:
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = 1
data.loc[data['FamilySize'] > 1,'IsAlone'] = 0
data.head()

,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,FamilySize,IsAlone
PassengerId,,,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,1,0,0,1,2,0
2,1,38.0,1,0,71.2833,1,0,0,1,0,0,2,0
3,1,26.0,0,0,7.9250,0,0,1,0,0,1,1,1
4,1,35.0,1,0,53.1000,1,0,0,0,0,1,2,0
5,0,35.0,0,0,8.0500,0,0,1,0,0,1,1,1


In [29]:
data_test['FamilySize'] = data_test['SibSp'] + data_test['Parch'] + 1
data_test['IsAlone'] = 1
data_test.loc[data_test['FamilySize'] > 1,'IsAlone'] = 0

,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,FamilySize,IsAlone
PassengerId,,,,,,,,,,,,,
892,0,34.5,0,0,7.8292,0,0,1,0,1,0,1,1
893,1,47.0,1,0,7.0000,0,0,1,0,0,1,2,0
894,0,62.0,0,0,9.6875,0,1,0,0,1,0,1,1
895,0,27.0,0,0,8.6625,0,0,1,0,0,1,1,1
896,1,22.0,1,1,12.2875,0,0,1,0,0,1,3,0


In [32]:
data.drop('Parch', axis=1, inplace=True)
data.drop('SibSp', axis=1, inplace=True)

In [33]:
data_test.drop('Parch', axis=1, inplace=True)
data_test.drop('SibSp', axis=1, inplace=True)

In [36]:
X_train, X_valid, y_train, y_valid = train_test_split(data, target)

In [55]:
parameters = [{'n_estimators': [350, 400, 450], 
               'max_features': np.arange(5, 12),
               'max_depth': np.arange(6, 12)}]

In [56]:
clf = GridSearchCV(estimator=RandomForestClassifier(), 
                   param_grid=parameters,
                   scoring='accuracy',
                   cv=6)

In [57]:
%%time
clf.fit(X_train, y_train)

CPU times: user 3min 58s, sys: 26.4 ms, total: 3min 58s
Wall time: 3min 58s


GridSearchCV(cv=6, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [350, 400, 450], 'max_features': array([ 5,  6,  7,  8,  9, 10, 11]), 'max_depth': array([ 6,  7,  8,  9, 10, 11])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [58]:
clf.best_params_

{'max_depth': 10, 'max_features': 11, 'n_estimators': 400}

In [59]:
y_pred = clf.predict(X_valid)
accuracy_score(y_pred, y_valid)

0.8071748878923767

In [61]:
y_test_pred = clf.predict(data_test)
y_test_pred = pd.DataFrame({'Survived': y_test_pred}, index=data_test.index)
y_test_pred.to_csv('submission_1.csv')

In [63]:
# Стало хуже..